In [1]:
!pip install opencv-contrib-python
!pip install mediapipe opencv-python

In [21]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [22]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b= np.array(b)
    c= np.array(c)
   
    
    radians= np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0]) #radients for perticular joints
    
    angle =np.abs(radians*180.0/np.pi)
    if angle>180.0:
        angle = 360-angle
    return angle 
    

In [23]:
imperfect = False

select_type = input('type the squat format:')

if select_type == 'imperfect':
    imperfect = True
cap = cv2.VideoCapture(0)

#squat counter
counter = 0
stage = None 


#setup mediapipe instance 
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened(): 
     ret,frame = cap.read() 
       # Recoloring the image to RGB
     image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) 
     image.flags.writeable = False 
        
        # make detection 
     result = pose.process(image) 
    
        # Recolor back to BGR
     image.flags.writeable = True 
     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
     
     #Extract Landmarks
     try:
        landmarks = result.pose_landmarks.landmark
        
        #get co-ordinates
        shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        knee_right = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        ankle_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        
        shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee_left = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle_left = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        
        
        #generate angle
        angle_1_right= calculate_angle(shoulder_right,hip_right,knee_right)
        angle_2_right = calculate_angle(hip_right,knee_right,ankle_right)
        
        angle_1_left= calculate_angle(shoulder_left,hip_left,knee_left)
        angle_2_left = calculate_angle(hip_left,knee_left,ankle_left)
        
        int_angle_1_right = int(angle_1_right)
        int_angle_2_right = int(angle_2_right)
        
        int_angle_1_left = int(angle_1_left)
        int_angle_2_left = int(angle_2_left)
        #visualize angle
        cv2.putText(image,str(int_angle_1_right),
                        tuple(np.multiply(hip_right,[640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,0.9,(255,255,255),2,cv2.LINE_AA
                   )
                   
        cv2.putText(image,str(int_angle_2_right),
                        tuple(np.multiply(knee_right,[640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,0.9,(255,255,255),2,cv2.LINE_AA
                   )
        cv2.putText(image,str(int_angle_1_left),
                        tuple(np.multiply(hip_left,[640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,0.9,(255,255,255),2,cv2.LINE_AA
                   )
                   
        cv2.putText(image,str(int_angle_2_left),
                        tuple(np.multiply(knee_left,[640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,0.9,(255,255,255),2,cv2.LINE_AA
                   ) 
       
        #normal squats
        if imperfect:
            
            if int_angle_1_right <=65 and int_angle_2_right<= 65 and int_angle_1_left <=65 and int_angle_2_left <= 65:
                
                stance = "Down"
                
            if int_angle_2_right >90 and int_angle_2_left >90 and stance =="Down":
                stance = "up"
                counter +=1
                print(counter)
            
            
        #perfect squats
        else:
        
            if  int_angle_1_right>=52 and int_angle_1_right<=57 and int_angle_2_right>=52 and int_angle_2_right<=57 and int_angle_1_left>=52 and int_angle_1_left<=57 and int_angle_2_left>=52 and int_angle_2_left<=57:
                stance = "Down"

            if int_angle_2_right >90 and int_angle_2_left >90 and stance =="Down":
                stance = "up"
                counter +=1
                print(counter)
        
        
        
        
        
     except:
        pass
     
     #setup repetation box
     cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)
     
     #Repetation Data 
     cv2.putText(image,'REPS',(15,12),
                 cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
     cv2.putText(image,str(counter),(10,60),
                 cv2.FONT_HERSHEY_SIMPLEX,2,(225,225,225),2,cv2.LINE_AA)
    
     #Render detections
     mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color = (245,117,66),thickness = 2, circle_radius = 5),
                              mp_drawing.DrawingSpec(color = (245,66,230),thickness = 2, circle_radius = 5)
                               )
                                                     
            
        
     
    
     cv2.imshow('Mediapipe Feed',image) 
     if cv2.waitKey(10) & 0xFF == ord('q'):
        break
   
    cap.release() 
    cv2.destroyAllWindows() 

type the squat format:imperfect
1
2
